### 🎯 任務說明：將「平行世界發想器」設計為 Chain-of-Thought（CoT）推理過程

輸入一件日常小事，想像它在不同平行世界中可能演變成的荒謬劇情，並生成一段生動的小故事。

### ✅ CoT 改寫版本流程：


1.   Planning（發散不同故事）：
請 LLM 思考七種「如果在不同平行世界，這件小事會發生怎樣荒謬發展」的劇情，每個劇情以 3-4 句描述。
2.   writing（編造故事情節）：
從這七個平行世界劇情中，挑選最荒謬有趣的一個，根據使用者選擇的故事風格（如搞笑版、熱血版、悲劇版、中二病版、溫馨療癒版），擴寫成第一人稱短篇故事。


#### 1. 讀入金鑰


In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "openai/gpt-oss-20b"

In [ ]:
!pip install aisuite[all]

### 2. 使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [ ]:
provider_planner = "groq"
model_planner = "gemma2-9b-it"

provider_writer = "groq"
model_writer = "gemma2-9b-it"


In [ ]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [ ]:
system_planner = "請用台灣習慣的中文回應：你是一位腦洞超大的平行世界作家，擅長根據日常事件想像（如科幻、魔法、奇幻、現代變異、超現實、搞笑...等）在不同平行世界中發生的荒謬劇情。請條列式列出七種，每一個劇情 3-4 句話描述即可。"

def get_system_writer(style):
    """
    根據選擇的風格，回傳不同的 system_writer prompt
    """
    return f"請用台灣習慣的中文回應：你是一位{style}的短篇小說家。請從五個平行世界劇情中選出一個最荒謬的，並將它擴寫成一段第一人稱小故事（大約100-200字），口吻要符合「{style}」的感覺，可以自由發揮細節和情緒。"


In [ ]:
def parallel_world_post(user_event, story_style):
    """
    根據使用者輸入的小事 + 選擇的故事風格，
    1. 思考平行世界版本
    2. 擴寫成符合風格的小故事
    """
    # Step 1: CoT - 想像平行世界
    planning_prompt = f"使用者說：{user_event}。請幫我想像七個在不同平行世界中，這件事情可能發生的荒謬發展。"
    parallel_scenarios = reply(
        system=system_planner,
        prompt=planning_prompt,
        provider=provider_planner,
        model=model_planner
    )

    # Step 2: 擴寫成故事（依照選擇的風格）
    system_writer = get_system_writer(story_style)
    generation_prompt = f"這是我想到的七個平行世界劇情：\n{parallel_scenarios}\n\n請從中選出一個最荒謬的劇情，然後寫成符合「{story_style}」風格的第一人稱小故事。"
    final_story = reply(
        system=system_writer,
        prompt=generation_prompt,
        provider=provider_writer,
        model=model_writer
    )

    return parallel_scenarios, final_story


### 4. 用 Gradio 打造對話機器人 Web App!

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
import gradio as gr
import random

styles = ["搞笑版", "熱血版", "悲劇版", "中二病版", "溫馨療癒版", "貓咪喵喵版"]
with gr.Blocks() as demo:
    gr.Markdown("### 🌌 平行世界發想器 Parallel Universe Explorer 🚀")
    gr.Markdown("請輸入一件日常小事，讓 AI 帶你穿越到七個天馬行空的平行世界！")

    user_input = gr.Textbox(label="今天發生的事情是…")

    story_style = gr.Radio(
        choices=styles + ["隨機選擇"],
        label="選擇你想要的故事風格",
        value="搞笑版",
        interactive=True
    )

    btn = gr.Button("天馬行空的想像開始啦 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 七個發生在不同平行宇宙的故事")
        out2 = gr.Textbox(label="📖 異想天開短篇故事")

    def wrapped_parallel_world_post(user_event, selected_style):
        """
        如果使用者選了「隨機選擇」，就隨機挑一個 style
        """
        if selected_style == "隨機選擇":
            selected_style = random.choice(styles)
        return parallel_world_post(user_event, selected_style)

    btn.click(wrapped_parallel_world_post, inputs=[user_input, story_style], outputs=[out1, out2])

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2fb01578c8c8d2f01d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
demo.launch(share=True, debug=True)